In [1]:
import glob
import pandas as pd
from classes import CTScan
from tqdm import tqdm

In [2]:
INPUT_FOLDER = r'D:/masters_datasets/LUNA16-original/LUNA16_Images/*.mhd'
paths = glob.glob(INPUT_FOLDER)
print(len(paths))
paths.sort()

888


In [3]:
RESOURCES_PATH = r'D:/masters_datasets/LUNA16-ann'
annotations = pd.read_csv(RESOURCES_PATH + '/annotations.csv')
candidates = pd.read_csv(RESOURCES_PATH + '/candidates.csv')
print(annotations.shape) 
print(candidates.shape) 

(1186, 5)
(551065, 5)


In [4]:
annotations.head()

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150


In [5]:
candidates.head()

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-56.08,-67.85,-311.92,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,53.21,-244.41,-245.17,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.66,-121.80,-286.62,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-33.66,-72.75,-308.41,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-32.25,-85.36,-362.51,0


In [6]:
def _get_positive_series(paths):
    file_list = [f.replace('\\', '/').split('/')[-1][:-4] for f in paths]
    series = annotations['seriesuid'].tolist()
    infected = [f for f in file_list if f in series]
    return infected

def _get_negative_series(paths):
    file_list = [f.replace('\\', '/').split('/')[-1][:-4] for f in paths]
    series = annotations['seriesuid'].tolist()
    cleans = [f for f in file_list if f not in series]
    return cleans

In [7]:
meta_data = pd.DataFrame(columns=['seriesuid', 'spacing', 'lungs_bounding_box', 'centers', 'radii', 'class'])

for series_id in tqdm(_get_positive_series(paths)): # n = 601
    nodule_coords_annot = annotations[annotations['seriesuid'] == series_id]
    tp_co = [(a['coordZ'], a['coordY'], a['coordX']) for a in nodule_coords_annot.iloc]
    radii = [(a['diameter_mm'] / 2) for a in nodule_coords_annot.iloc]
    ct = CTScan(seriesuid=series_id, centers=tp_co, radii=radii, clazz=1)
    ct.preprocess()
    # ct.save_preprocessed_image()
    diction = ct.get_info_dict()
    # meta_data = meta_data.append(pd.Series(diction), ignore_index=True)
    meta_data.loc[len(meta_data)] = pd.Series(diction)

meta_data.to_csv('preprocessed_meta.csv', index=False)

100%|██████████| 601/601 [38:51<00:00,  3.88s/it]


In [8]:
# content_size = 0
# for series_id in _get_negative_series(paths): # n = 287
#     content_size += 1

# print(content_size)

In [9]:
INPUT_FOLDER = r'C:/Users/ejrza/Downloads/mestrado/projeto/code/masters-object/LungSegmentation/output/*.nrrd'
file_list = glob.glob(INPUT_FOLDER)
print(len(file_list))
file_list.sort()

888


In [10]:
### continue to copy and reading the code from prepare/run_preprocess.py